# 💸 markowitz 

diego domenzain

5.2025

In [8]:
# ----------------------------------------------------------------------------
#
#                              📚📚📚📚
#
# ----------------------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
# this guy below uncommented makes plots pop out the notebook
%matplotlib tk
# ----------------------------------------------------------------------------

In [9]:
# • markowitz targets
# ----------------------------------------------------------------------------
# how much to invest?
mu = 100
# how much to earn?
rho = 0.5*mu
# ----------------------------------------------------------------------------


In [10]:
# • data in
# ----------------------------------------------------------------------------
ntimes, nstocks = 252, 3
np.random.seed(42)

# • geometric Brownian Motion with drift
#   muu .....daily drift (e.g., 7% annualized)
#   sigmaa ..daily volatility (15% annualized)
#   soo .....starting price for ^GSPC
muu = np.asarray([0.07, 0.03, 0.05]) / 252
sigmaa = np.asarray([0.15, 0.2, 0.3]) / np.sqrt(252)
soo = np.asarray([4000, 2000, 1234])

stockos = np.zeros((ntimes,nstocks),dtype=np.float64)

for istock in range(0,nstocks):
  returns = np.random.normal(loc=muu[istock], scale=sigmaa[istock], size=ntimes)
  stockos[:,istock] = soo[istock] * np.exp(np.cumsum(returns))
# ----------------------------------------------------------------------------
# 
# 🕸️ fetch from the web
# ----------------------------------------------------------------------------
# import yfinance as yf
# import pandas as pd

# # • lots of tickers
# url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
# table = pd.read_html(url)
# tickers = table[0]['Symbol'].tolist()

# tickers = tickers[:20]

# # • times
# start_date = '2018-01-01'
# end_date = '2023-01-01'

# # • magic (data in usd)
# yfdata = yf.download(tickers, start=start_date, end=end_date)['Close']
# yfdata = yfdata.dropna()
# stockos = yfdata.to_numpy()
# ntimes = stockos.shape[0]
# nstocks = len(tickers)

# dates = yfdata.index.to_list()
# ---------------------------------------------------------------------------

In [11]:
# • build some boring stuff
# ----------------------------------------------------------------------------
stocknum = np.zeros((nstocks,),dtype=np.int32)
for istock in range(0,nstocks):
  stocknum[istock] = istock

tiempo = np.zeros((ntimes,),dtype=np.float64)
for itiempo in range(0,ntimes):
  tiempo[itiempo] = itiempo
# ----------------------------------------------------------------------------

In [12]:
# • the whole schebang
# ----------------------------------------------------------------------------
# 
# 👷👷
# 
# ----------------------------------------------------------------------------
S = np.cov(stockos.T)        # of size nstocks × nstocks
r = np.mean(stockos, axis=0) # of size nstocks
rhomiu = np.asarray([rho,mu],dtype=np.float64)
# ----------------------------------------------------------------------------
# 
# A:                w:     b:
# 
#   [ 2⋅Σ -r -e ]   [ω ]   [ 0 ]
#   [ rᵀ   0  0 ] * [l₁] = [ ρ ]
#   [ eᵀ   0  0 ]   [l₂]   [ μ ]
# 
# ----------------------------------------------------------------------------
A = np.zeros((nstocks+2,nstocks+2),dtype=np.float64)
# 2⋅Σ
for ir in range(0,nstocks):
  for ic in range(0,nstocks):
    A[ir,ic] = 2*S[ir,ic]
# -r
ic = nstocks
for ir in range(0,nstocks):
  A[ir,ic] = -r[ir]
# -e
ic = nstocks+1
for ir in range(0,nstocks):
  A[ir,ic] = -1.0
# rᵀ
ir = nstocks
for ic in range(0,nstocks):
  A[ir,ic] = r[ic]
# eᵀ
ir = nstocks+1
for ic in range(0,nstocks):
  A[ir,ic] = 1.0
# ----------------------------------------------------------------------------
# b:
#   [ 0 ]
#   [ ρ ]
#   [ μ ]

b = np.zeros((nstocks+2,),dtype=np.float64)
for ic in range(0,2):
  b[nstocks+ic] = rhomiu[ic]
# ----------------------------------------------------------------------------
# AᵀA·w = Aᵀ·b
#     w = np.linalg.solve(AᵀA,  Aᵀ·b)
B = np.dot(A.T,A)
# B ⟵ B + β⋅Id
bb = np.dot(A.T,b)
# ----------------------------------------------------------------------------
# 
# 🎸💃
# 
# ----------------------------------------------------------------------------
w = np.linalg.solve(B,bb)
# ----------------------------------------------------------------------------
# 💾
omeg = np.zeros((nstocks,),dtype=np.float64)
for ii in range(0,nstocks):
  omeg[ii] = w[ii]
# ----------------------------------------------------------------------------

In [13]:
# • 🖨️
# ----------------------------------------------------------------------------
e_ = np.ones((nstocks,),dtype=np.float64)

E1 = np.dot(omeg, np.dot(S, omeg))
E2 = np.dot(r, omeg)-rho
E3 = np.dot(e_, omeg)-mu
# ----------------------------------------------------------------------------
print(f"  the objective functions gave:\n   Φ risk (should be small)       {E1:f} \n   Φ return (should be zero)      {E2:f} \n   Φ sum-to-one (should be zero)  {E3:f}\n\n")
# ----------------------------------------------------------------------------
print(f" asset # | weight value")
print(f" --------.------------")
for istock in range(0,nstocks):
  print(f"    {istock:d}    |    {omeg[istock]:2.2f}")
# ----------------------------------------------------------------------------

  the objective functions gave:
   Φ risk (should be small)       161918853.142165 
   Φ return (should be zero)      0.000000 
   Φ sum-to-one (should be zero)  -0.000081


 asset # | weight value
 --------.------------
    0    |    -35.68
    1    |    0.34
    2    |    135.34


In [14]:
# • plot
# ----------------------------------------------------------------------------
# 🔵🟠🟢🔴
fig, ax = plt.subplots(1,2)

for istock in range(0,nstocks):
  ax[0].plot(tiempo, stockos[:,istock])
  ax[1].scatter(stocknum[istock],omeg[istock])

ax[0].set_aspect(1.0 / ax[0].get_data_ratio())
ax[1].set_aspect(1.0 / ax[1].get_data_ratio())

ax[0].set_xlabel("Time (?)")
ax[0].set_ylabel("Stock value (?)")

ax[1].set_xlabel("Stock num.")
ax[1].set_ylabel("Weight value (?)")

plt.show()
# ----------------------------------------------------------------------------

**when to sell? 🤔**

In [15]:
# ----------------------------------------------------------------------------
def monte_carlo_holding_period(stockos, omeg, mu, rho, alpha=0.95, nsimu=10000, tstepmax=60):
  """
  estimate minimum holding period such that P(Cumulative return >= rho) >= alpha

  stockos ...(ntimes x nstocks) array of historical prices
  omeg ......(nstocks,) array of portfolio omeg
  mu ........initial investment (USD)
  rho .......target return (USD)
  alpha .....desired confidence level
  nsimu .....number of Monte Carlo paths to simulate
  tstepmax ..max number of time steps to evaluate

  returns:
  minimum time t (in time steps) satisfying the confidence condition
  """
  
  # • timely log returns
  log_returns = np.log(stockos[1:] / stockos[:-1])
  
  # • portfolio returns: weighted sum
  #   port_returns ... array of size ntimes-1
  port_returns = np.dot(log_returns,omeg)

  t_steps = np.arange(1, tstepmax + 1)

  for t in t_steps:
    # • t-step cumulative returns
    #   sim_returns ... matrix of size nsimu by t-steps
    #   each row is a random realization of returns over t steps
    sim_returns = np.random.choice(port_returns, size=(nsimu, t), replace=True)
    
    # • returns over all t-steps for each random realization
    cum_returns = np.sum(sim_returns, axis=1)

    # • multiplicative returns and USD gains
    total_gains = mu * (np.exp(cum_returns) - 1)

    # • check if confidence condition is satisfied
    prob = np.mean(total_gains >= rho)

    if prob >= alpha:
      return t

  return None
# ----------------------------------------------------------------------------

In [16]:
nsimu=10000
tstepmax=1
alpha = 0.95

tstar = monte_carlo_holding_period(stockos, omeg, mu, rho, alpha)
# ----------------------------------------------------------------------------
print(f"⏲️ minimum holding period to reach ${rho} with {int(alpha*100)}% confidence: {tstar} time steps")
# ----------------------------------------------------------------------------

⏲️ minimum holding period to reach $50.0 with 95% confidence: None time steps


                  fin